In [1]:
#Cargamos los módulos que se van a necesitar
import pandas as pd

import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')  # Cambia el backend a Qt5Agg para mostrar ventanas gráficas
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist
from pymoo.core.problem import ElementwiseProblem

from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_score

from matplotlib import style

#Para que el primer punto sea el inicial
from pymoo.core.repair import Repair 

#Diferentes funciones de la biblioteca para optimizar (pymoo)
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.problems.single.traveling_salesman import create_random_tsp_problem
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.operators.mutation.bitflip import BitflipMutation

In [2]:
#En primer lugar cargamos todos los archivos necesarios. Estos son los siguientes:
#1- Datos.txt -> Nos dice si queremos mapa o no, número de segmentos y número de puntos de control
#2- Coordenadas.txt -> Nos dice las coordenadas de los puntos de control
#3- flight_plan -> Nos da todos los datos de lo que se desea en la ruta (Puntos de paso, regiones...)

#flightplan_entrada = input("Introduce el archivo que quiere optimizar con la extensión .xml: ")
#flightplan_salida = input("Introduce como quiere llamar archivo de salida con .xml: ")

Estrategia = pd.read_csv('/home/dacya-iagesbloom/Desktop/PprzGCS/datos.txt', delimiter=':', header = None) #Datos.txt
Coordenadas = pd.read_csv('/home/dacya-iagesbloom/Desktop/PprzGCS/coordenadas.txt', delimiter=',', header=None) #Coordenadas.txt

#Carga de archivo XML
import xml.etree.ElementTree as ET
from xml.dom import minidom

tree = ET.parse(f'/home/dacya-iagesbloom/Documents/paparazzi/conf/flight_plans/prueba2.xml')
root = tree.getroot()

# Encontrar todos los waypoints
way_points = root.find('waypoints')
waypoint_list = []
squares_list = []
unchanged_points = []

# Almacenar los waypoints existentes en una lista
for waypoint in way_points.findall('waypoint'):
    name = waypoint.get('name')
    x = float(waypoint.get('x'))
    y = float(waypoint.get('y'))
    if name.startswith('wp'):
        waypoint_list.append((name, x, y))
    else:
        unchanged_points.append((name, x, y))
way_points.clear()
#PConvertimos tanto la lista de waypoints como unchanged en una lista y las unimos
waypoints = np.array([[nombre, x, y] for nombre, x, y in waypoint_list])
unchangedpoints = np.array([[nombre, x, y] for nombre, x, y in unchanged_points])
points = np.concatenate((waypoints, unchangedpoints), axis=0)


#Definimos una matriz unicamente con las coordenadas de cada parada:
paradas = (waypoints[:,-2:])
print(paradas)
#Definimos parámetros del barco
velocidad_media = 40

[['50.0' '-92.3']
 ['156.4' '212.8']
 ['-146.2' '22.3']
 ['-250.1' '255.3']
 ['-329.4' '-153.1']
 ['-66.8' '-444.9']
 ['199.3' '13.3']
 ['-47.5' '-233.4']]
